In [ ]:
from viewscad import Renderer

r = Renderer()
r.width = 700
r.height = 700

render = r.render


In [ ]:
from utils import ring, connection, pipe, cutout, two_sided_connection
from solid import cylinder, translate, rotate, cube
from solid.utils import *
from viewscad import Renderer

r = Renderer()
r.width = 700
r.height = 700

render = r.render


def bottom_grid(d=60, w=2, hole_size=6, hole_gap=2):
    plate = cylinder(h=w, d=d, segments=100)
    hole = cylinder(h=w, d=hole_size, segments=10)
    pane = range(-d//2, d//2, hole_size+hole_gap)
    transition = ring(d1=d, h=w, w=w*2)

    holes = [translate((x, y, 0))(hole) for x in pane for y in pane]

    return plate - holes + transition


def spin_around_tower(obj, count=4, alpha=45, h=160):
    angles = range(180, 540, int(360/count))
    # drehen und dann kippen
    return translate((0, 0, h/2))([rotate((alpha, 0, z))(obj) for z in angles])


def cup_con(d1=52, d2=60, w=2, height=80, inlet_delta=5):
    c = d2
    gradient = (d2-d1)/height
    inlet = (gradient*inlet_delta+w) * 2
    socket_d1_delta = d1 + inlet
    socket_d2_delta = d2 + inlet
    c = cutout(c)

    base = ring(d1=socket_d1_delta, d2=socket_d2_delta, h=height, w=w)
    socket = base - c


    cup = ring(d1=d1, d2=d2, w=w, h=height)
    cup += bottom_grid(d=d1, w=w)

    outline = cylinder(d1=d1, d2=d2, h=height*2, segments=100)

    return (socket, cup, outline)


def aero_tank(d=180, height=180, wall=2, socket_h=20, tooth=4):
    body = ring(d1=d, h=height, w=wall)
    bottom = cylinder(d=d, h=wall, segments=100)
    (_, socket) = connection(d=d, h=socket_h,
                             wall=wall, tooth_depth=tooth, tooth_height=tooth*2)

    return bottom + body + up(height-socket_h)(socket)



def aero_tower(w=2, h=160, d=160, socket_h=20, pipe_count=1, alpha=45, cup_d = 60, cup_h=80, tooth=4):
    aero_tower = pipe(d=d, h=h, w=w, tooth=tooth, socket_h=socket_h)
    (cup_s, cup, cup_outline) = cup_con(d1=cup_d-cup_h//10,d2=cup_d, w=w, height=cup_h, inlet_delta=5)
    outline = cylinder(d=d, h=d, segments=100)
    holes = spin_around_tower(cup_outline, count=pipe_count, alpha=alpha)
    sockets = spin_around_tower(
        translate((0, 0, h/2+w))(cup_s), h=d, count=pipe_count, alpha=alpha)

    holes = translate((0, 0, -h/2))(holes)
    sockets = translate((0, 0, -h/2))(sockets)
    aero_tower = aero_tower - holes + sockets
    aero_tower *= outline

    tank = aero_tank(d=180, height=180, wall=w, tooth=tooth)
    adapter = two_sided_connection(outer_d=180, outer_w=w, outer_tooth=tooth,  height=socket_h, inner_d=d, inner_w=w, inner_tooth=tooth)

    return (aero_tower, cup, tank, adapter)


(tower, cup, tank, adapter) = aero_tower(pipe_count=5)
render(tower, outfile='./outfiles/aero_tower.stl')
render(cup, outfile='./outfiles/plant_cup.stl')
render(tank, outfile='./outfiles/aero_tower_tank.stl')
render(adapter, outfile='./outfiles/aero_tower_tank_adapter.stl')


In [6]:

from utils import pipe, ring, connection, mount_to_bottom, tended_obj, cutout, sliding_connector
from solid import cylinder, translate, rotate, cube, hull
from solid.utils import *
from viewscad import Renderer
from math import sqrt

r = Renderer()
r.width = 700
r.height = 700

render = r.render


def tower_head(d=160, w=2, tooth=4, socket_h=20, h=100, gap=0.1):
    delta_r = w + tooth + gap
    base_d = d-delta_r*2
    filter_d = 8
    filter_l = 100
    piezo_d = 20
    piezo_h = 3
    slider_l = 30
    slider_w = 20
    pump_d = 27.4
    pump_hole_d = pump_d + 2
    pump_hole_delta = -((pump_d + slider_l)/2 + w*2)

    # head body
    body = pipe(d=d, w=w, tooth=tooth, socket_h=socket_h, h=h)

    base = cylinder(d=base_d, h=w, segments=30)
    (slider_s, slider_a) = sliding_connector(
        length=slider_l, width=slider_w, gap=0.05)

    cable_hole = translate((-d/2, 0, h/2))(
        rotate((0, 90, 0))(
            cylinder(d=4, h=w, segments=30)
        )
    )

    pump_hole = rotate(0)(
        translate((pump_hole_delta, 0, 0))(
            cylinder(d=pump_hole_d, h=20, segments=30))
    )

    head = body + base - pump_hole - cable_hole - pump_hole - slider_s

    # water tank

    tank = down(piezo_h)(
        cube((slider_l, piezo_d+4, piezo_d+4), center=True)
        + translate((-slider_l/2, 0, (piezo_d+4)/2))(slider_a)
    )

    tank += right((slider_l+piezo_h)/2)(rotate((0, 90, 0))(
        hull()(
            cylinder(d=piezo_d+4, h=piezo_h+2, center=True, segments=30),
            translate((piezo_h, 0, -(piezo_h)/2))(
                cylinder(d=piezo_d+4, h=.1, center=True, segments=30))
        )
        - cylinder(d=piezo_d, h=piezo_h, center=True, segments=30)
        - cylinder(d=piezo_d-4, h=piezo_h+2, center=True, segments=30)
        - translate((-2*piezo_d, -piezo_d, -piezo_h)
                    )(cube((piezo_d*2, piezo_d*2, piezo_h*2)))
    ))

    tank -= down(4)(rotate((0, 80, 0))(
        cylinder(d=filter_d, h=filter_l, center=True, segments=30)
    ))

    tank += translate((-(slider_l+pump_d/2)/2, 0, -piezo_h))(
        cube((pump_d/2, piezo_d+4, piezo_d+4), center=True)
        - translate((1, 0, 2))(
            cube((pump_d/2-2, piezo_d, piezo_d+2), center=True)
        )
        - translate((0, 0, piezo_d*0.75))(
            cube((pump_d/2, piezo_d+4, piezo_d+2), center=True)
        )
    )

    # head lid

    head_lid_adapter_inner_d = d-(w+gap+tooth)*2
    head_lid_body = cylinder(d=head_lid_adapter_inner_d, segments=30, h=w)
    handle_base_thickness = 10
    handle_top_thickness = handle_base_thickness + (socket_h-w)*2

    (head_lid_adapter, _) = connection(d=d, h=socket_h,
                                       wall=w, tooth_depth=tooth, tooth_height=2*tooth, gap=0.2)

    head_lid_body += hull()(
        up(w/2)(cube((handle_base_thickness, head_lid_adapter_inner_d, w), center=True)),
        up((socket_h-w/2))(cube((handle_top_thickness,
                                 head_lid_adapter_inner_d, w), center=True)),
    ) * cylinder(d=head_lid_adapter_inner_d, segments=30, h=socket_h)

    head_lid = head_lid_adapter + head_lid_body

    return (head, tank, head_lid)


(head, tank, head_lid) = tower_head()

render(head, outfile='outfiles/aero_tower_head.stl')
render(tank, outfile='outfiles/aero_tower_head_tank.stl')
render(head_lid, outfile='outfiles/aero_tower_head_lid.stl')


Geometries in cache: 21
Geometry cache size in bytes: 138408
CGAL Polyhedrons in cache: 20
CGAL cache size in bytes: 14148072
Total rendering time: 0:00:22.811
   Top level object is a 3D object:
   Simple:         no
   Vertices:     2101
   Halfedges:    6934
   Edges:        3467
   Halffacets:   2384
   Facets:       1192
   Volumes:         3
EXPORT-WARNING: Exported object may not be a valid 2-manifold and may need repair


Geometries in cache: 22
Geometry cache size in bytes: 57920
CGAL Polyhedrons in cache: 14
CGAL cache size in bytes: 1775512
Total rendering time: 0:00:02.047
   Top level object is a 3D object:
   Simple:        yes
   Vertices:      269
   Halfedges:     822
   Edges:         411
   Halffacets:    292
   Facets:        146
   Volumes:         3


Geometries in cache: 11
Geometry cache size in bytes: 57832
CGAL Polyhedrons in cache: 7
CGAL cache size in bytes: 3789568
Total rendering time: 0:00:06.426
   Top level object is a 3D object:
   Simple:        yes
   Vertices:     1206
   Halfedges:    3842
   Edges:        1921
   Halffacets:   1240
   Facets:        620
   Volumes:         3
